Updating stations in OSCAR/Surface
---
> **Example #3:** We will update the schedules of German stations. We want to update stations that only have a _30 min reporting interval_ . Their reporting interval  should be changed to 1h and the international exchange flag set to _true_. In case there is no information on start and endtime of the schedule, set it to correspond to a 7/24 schedule.

***Note***: we limit ourselves to observations tracked by WDMQS for the purpose of this exercise. We thus look for **ongoing** observations of _pressure, temperature, wind_ and _humidity_ at stations.




In [1]:
import json
import sys
import logging
logging.basicConfig(level=logging.INFO)

from oscar_lib import OscarClient, extractSchedules, Station

wdqms_variables = [216,210,224,251,309,12005,12006] # pressure,precipitation,temperature,humidity,wind deprecated, wind speed, wind direction

In [2]:
client = OscarClient(oscar_url = OscarClient.OSCAR_DEPL, token="changeMe")
stations=client.oscarSearch({'territoryName':'DEU', 'facilityType':'landFixed'})

INFO:oscar_lib.oscar_client:searching for {'territoryName': 'DEU', 'facilityType': 'landFixed'} at https://oscardepl.wmo.int/surface//rest/api/search/station


In [3]:
# print out the first search result
stations["data"][0] 

{'id': 12494,
 'name': 'FILCHNER',
 'region': 'Antarctica',
 'territory': 'Germany',
 'declaredStatus': 'Operational',
 'latitude': -80.43681,
 'longitude': -44.43199,
 'elevation': 103,
 'stationTypeName': 'Land (fixed)',
 'wigosStationIdentifiers': [{'wigosStationIdentifier': '0-20000-0-89047',
   'primary': True}],
 'wigosId': '0-20000-0-89047',
 'stationTypeId': 1,
 'dateEstablished': '2016-01-14T23:00:00.000+00:00',
 'stationStatusCode': 'operational',
 'stationTypeCode': 'landOceanSurface',
 'stationProgramsDeclaredStatuses': 'GOS:Operational'}

In [4]:
# collect primary wigos ids from search result. 
# Need to loop through search result and potential multiple wigos ids to find the primary one
german_station_wigosids = [ wid["wigosStationIdentifier"] for station in stations["data"] for wid in station["wigosStationIdentifiers"] if wid["primary"] ]

#print out first 5 entries
german_station_wigosids[0:5]

['0-20000-0-89047',
 '0-20000-0-89003',
 '0-20000-0-89002',
 '0-20008-0-NMY',
 '0-20000-0-89011']

In [5]:
limit=10
station_map = {}
for wigos_id in german_station_wigosids[0:min(limit,len(german_station_wigosids))]:
    try:
        print("loading ",wigos_id)
        myxml = client.load_station(wigos_id=wigos_id)
        s = Station(myxml)
        
        if "20008" in wigos_id: # ignore GAWSIS stations
            continue
        
        # check if the station has at least one ongoing 30 min schedule (for WDQMS relevant variables)
        schedules = s.current_schedules(variables=wdqms_variables)

        # loop through nested datastructure and check if interval is 30min
        intervals = [ dg["schedule"]["interval"] == "PT30M" for o in schedules.values() for d in o["deployments"] for dg in d["datagenerations"] ]

        if True or any(intervals): # we consider the station as long as there is one 30min schedule
            station_map[wigos_id]=s
        else:
            print(wigos_id,"has no 30 min schedule")
    
    except Exception as e:
        print("error loading station {} {}".format(wigos_id,str(e)))


loading  0-20000-0-89047
loading  0-20000-0-89003
loading  0-20000-0-89002
loading  0-20008-0-NMY
loading  0-20000-0-89011
loading  0-20000-0-10505
loading  0-20000-0-10309
loading  0-20000-0-10442
loading  0-20000-0-10954
loading  0-20000-0-10520


Output a schedule of a station. 251 stands for [humidity](https://codes.wmo.int/wmdr/ObservedVariableAtmosphere/_251)

In [6]:
# output a schedule of a station. 251 is 
print(json.dumps(list(station_map.values())[0].current_schedules()[251],indent=1))

{
 "gid": null,
 "variableid": "http://codes.wmo.int/wmdr/ObservedVariableAtmosphere/251",
 "deployments": [
  {
   "gid": "id_0c244c02-3db2-40b2-a2fa-94503e59ee58",
   "from": "2016-04-29T00:00:00Z",
   "to": null,
   "datagenerations": [
    {
     "gid": "id_bc41c15c-0697-44a5-8430-99fc866c40e0",
     "schedule": {
      "from": "2016-04-29T00:00:00Z",
      "to": null,
      "startMonth": 1,
      "endMonth": 12,
      "startWeekday": 1,
      "endWeekday": 7,
      "startHour": 0,
      "endHour": 23,
      "startMinute": 0,
      "endMinute": 59,
      "interval": 3600,
      "international": true
     }
    }
   ]
  }
 ]
}


### update strategy: 
 1. set 1h international 7/24 schedule for "ongoing" schedules. 
 2. other schedules will be merged with the same 1h international 7/24 schedule, preserving the elements present in the original schedule



In [7]:
defaultSchedule = { 
    "from": "2016-04-28T00:00:00Z", "to" : None,     
    "startMonth": 1,"endMonth": 12,
    "startWeekday": 1, "endWeekday": 7,
    "startHour": 0, "startMinute": 0,          
    "endHour": 23,    "endMinute": 59,
    "interval": 60*60,     "international": True   
}


In [8]:
for wigos_id,station in station_map.items():
    print("updating station",wigos_id)
    
    current_schedules = station.current_schedules() # get "ongoing" schedules
    for current_schedule in current_schedules.values():
        for d in current_schedule["deployments"]:
            for dg in d["datagenerations"]:
                gid=dg["gid"]
                schedule=dg["schedule"]
                
                station.fix_and_update_datageneration(gid,defaultSchedule) 

    try:
        station.validate()
        print("validates ok")
    except: # if there are remaining validation issues
        print("fixing remaining schedules")
        station.fix_deployments(mode="update",defaultSchedule=defaultSchedule) #merge remaining eronous schedules

    station.validate()
    
    try:
        station.save(client)
        print("saved",wigos_id)
    except Exception as e:
        print("station could not be saved:",e)
    

updating station 0-20000-0-89047
validates ok


INFO:oscar_lib.oscar_client:upload ok, new id 196351 The list below is organized by section header and shows exceptions/issues – if any – that may have resulted from the processing of the XML (NB: Section headers are always displayed).
# Facility with identifier "0-20000-0-89047"
REF_6: No value found for the element "Equipment/geospatialLocation/../beginPosition". Information for this item is discarded.
REF_6: No value found for the element "Equipment/geospatialLocation/../beginPosition". Information for this item is discarded.
REF_6: No value found for the element "Equipment/geospatialLocation/../beginPosition". Information for this item is discarded.
REF_6: No value found for the element "Equipment/geospatialLocation/../beginPosition". Information for this item is discarded.

INFO:oscar_lib.oscar_client:status: VALID_XML_WITH_ERRORS_OR_WARNINGS message: upload ok, new id 196351 The list below is organized by section header and shows exceptions/issues – if any – that may have resulte

station could not be saved: error saving original XML (VALID_XML_WITH_ERRORS_OR_WARNINGS)
updating station 0-20000-0-89003
validates ok


INFO:oscar_lib.oscar_client:upload ok, new id 196352 The list below is organized by section header and shows exceptions/issues – if any – that may have resulted from the processing of the XML (NB: Section headers are always displayed).
# Facility with identifier "0-20000-0-89003"
REF_4: The alias "HALVFARRYGGEN EP11" is already assigned to this station . This alias is discarded.
REF_6: No value found for the element "Equipment/geospatialLocation/../beginPosition". Information for this item is discarded.
REF_9: A mandatory element is not provided. The "Equipment/geospatialLocation/Point/pos" is discarded.
REF_9: A mandatory element is not provided. The "Equipment/geospatialLocation/Point/pos" is discarded.
REF_9: A mandatory element is not provided. The "Equipment/geospatialLocation/Point/pos" is discarded.

INFO:oscar_lib.oscar_client:status: VALID_XML_WITH_ERRORS_OR_WARNINGS message: upload ok, new id 196352 The list below is organized by section header and shows exceptions/issues – i

station could not be saved: error saving original XML (VALID_XML_WITH_ERRORS_OR_WARNINGS)
updating station 0-20000-0-89002
validates ok


...

saved 0-20000-0-10442
updating station 0-20000-0-10954
validates ok


INFO:oscar_lib.oscar_client:upload failed with status: REJECTED the log is The list below is organized by section header and shows exceptions/issues – if any – that may have resulted from the processing of the XML (NB: Section headers are always displayed).
REF_2: No inserts or updates have been made by the received XML: before resubmitting the file please read and then remove this comment at the top of the XML: "The element wmdr:internationalExchange has been exported as 'false' (real value is 'null'). Before resubmitting this XML please remove this comment and review the value of wmdr:internationalExchange under wmdr:DataGeneration with gml:id id_2529129d-bf23-41fa-9edf-c2c34a0c4938, id_6976f092-bd7d-4fad-8b80-8998739ffb4a, id_88a28262-b7de-4993-b3c0-4c5d5bcfe129, id_64cdf214-387e-4311-a1c4-783d3739ff54, id_651bad2f-b0c3-4c4c-b4aa-8b32d95c5a55, id_b074e82a-68a7-7937-e053-148ef98d6de4, id_b074e82a-68a8-7937-e053-148ef98d6de4, id_b074e82a-68a9-7937-e053-148ef98d6de4, id_6de557d9-233a-4

station could not be saved: error saving original XML (REJECTED)
updating station 0-20000-0-10520
validates ok


INFO:oscar_lib.oscar_client:upload ok, new id 196363 
INFO:oscar_lib.oscar_client:status: SUCCESS message: upload ok, new id 196363  
INFO:oscar_lib.oscar_client:upload ok, new id 196364 The list below is organized by section header and shows exceptions/issues – if any – that may have resulted from the processing of the XML (NB: Section headers are always displayed).
# Facility with identifier "0-20000-0-10520"
REF_3 The element "Process/deployment/Deployment/validPeriod" is discarded. Deployment's valid period is filled in based on maximum extent of existing data generations.
REF_3 The element "Process/deployment/Deployment/validPeriod" is discarded. Deployment's valid period is filled in based on maximum extent of existing data generations.
REF_3 The element "Process/deployment/Deployment/validPeriod" is discarded. Deployment's valid period is filled in based on maximum extent of existing data generations.
REF_3 The element "Process/deployment/Deployment/validPeriod" is discarded. De

saved 0-20000-0-10520
